In [2]:
import pandas as pd
#from requests_html import HTMLSession, AsyncHTMLSession
from selenium.webdriver.chrome.options import Options

from selenium import webdriver
from selenium.webdriver.common.by import By
import re

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [4]:
%%time
def parse_cian(url, pages_num, filename):
    stats = []
    
    #PROXY = "http://proxy.example.com:8080"

    options = Options()
    #options.add_argument('--headless')
    #PROXY = "84.239.49.201:3128" # IP:PORT or HOST:PORT
    options.add_argument("--blink-settings=imagesEnabled=false")
    #options.add_argument('--proxy-server=%s' % PROXY)

    driver = webdriver.Chrome(options=options)
    driver.set_window_position(-2000, 0)
    driver.get(url)

    for i in range(1, pages_num+1):
        links = driver.find_elements(by=By.CLASS_NAME, value="_93444fe79c--general--BCXJ4")

        pages = webdriver.Chrome(options=options)
        pages.set_window_position(-2000, 0)
        pages.get('about:blank')

        for i, link in enumerate(links):
            pages.execute_script(f"window.open('{link.find_elements(by=By.CLASS_NAME, value='_93444fe79c--link--eoxce')[0].get_attribute('href')}','new_window{i}')")

        for i in range(len(links)):
        #for i in range(1, 2):
            try:
                apart = {}
                pages.switch_to.window(pages.window_handles[i])
                try:
                    apart['price'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--price_value--lqIK0").text.replace('₽', '').replace(' ', '')
                    try:
                        metro = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--undergrounds--vmBnV")
                        apart['metro'] = min([int(x) for x in re.sub(r'[^\d\s]+', '', metro.text).replace('\n', '').split(' ') if x])
                    except:
                        apart['metro'] = None
                    try:
                        apart['address'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--address--F06X3").text.replace('На карте', '')
                    except:
                        apart['address'] = None
                    try:
                        apart['district'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--address--F06X3").find_elements(by=By.TAG_NAME, value='a')[2].text.replace('р-н ', '')
                    except:
                        apart['district'] = None
                    try:
                        apart['rooms'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--title--UEAG3").text.split('-')[0]
                    except:
                        apart['rooms'] = None
                    try:
                        apart['total_area'] = pages.find_element(by=By.XPATH, value="//div[contains(text(), 'Общая')]").find_element(by=By.XPATH, value="preceding-sibling::div").text.replace(' м²', '')
                    except:
                        apart['total_area'] = None

                    try:
                        apart['live_area'] = pages.find_element(by=By.XPATH, value="//div[contains(text(), 'Жилая')]").find_element(by=By.XPATH, value="preceding-sibling::div").text.replace(' м²', '')
                    except:
                        apart['live_area'] = None
                    try:
                        apart['kitchen_area'] = pages.find_element(by=By.XPATH, value="//div[contains(text(), 'Кухня')]").find_element(by=By.XPATH, value="preceding-sibling::div").text.replace(' м²', '')
                    except:
                        apart['kitchen_area'] = None
                    try:
                        apart['floor'] = pages.find_element(by=By.XPATH, value="//div[contains(text(), 'Этаж')]").find_element(by=By.XPATH, value="preceding-sibling::div").text.split(' из ')[0]
                        apart['max_floor'] = pages.find_element(by=By.XPATH, value="//div[contains(text(), 'Этаж')]").find_element(by=By.XPATH, value="preceding-sibling::div").text.split(' из ')[1]
                    except:
                        apart['floor'] = None
                        apart['max_floor'] = None
                    try:
                        apart['build_year'] = pages.find_element(by=By.XPATH, value="//div[contains(text(), 'Год постройки')]").find_element(by=By.XPATH, value="following-sibling::div").text
                    except:
                        apart['build_year'] = None
                    try:
                        apart['housing_type'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Тип жилья')]").find_element(by=By.XPATH, value="following-sibling::span").text
                    except:
                        apart['housing_type'] = None
                    try:
                        apart['meter_cost'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--price_per_meter--yfcbi").text.replace(' ₽/м²', '').replace(' ', '')
                    except:
                        apart['meter_cost'] = None
                    try:
                        apart['bath'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Санузел')]").find_element(by=By.XPATH, value="following-sibling::span").text
                    except:
                        apart['bath'] = None
                    try:
                        apart['repair'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Ремонт')]").find_element(by=By.XPATH, value="following-sibling::span").text
                    except:
                        apart['repair'] = None
                    try:
                        apart['window'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Вид из окон')]").find_element(by=By.XPATH, value="following-sibling::span").text
                    except:
                        apart['window'] = None
                    try:
                        apart['ceiling'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Высота потолков')]").find_element(by=By.XPATH, value="following-sibling::span").text.replace(' м', '')
                    except:
                        apart['ceiling'] = None
                    try:
                        apart['description'] = pages.find_element(by=By.ID, value="description").text
                    except:
                        apart['description'] = None

                except:
                    apart['price'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--amount--ON6i1").text.replace('₽', '').replace(' ', '')
                    try:
                        metro = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--undergrounds--sGE99")
                        apart['metro'] = min([int(x) for x in re.sub(r'[^\d\s]+', '', metro.text).replace('\n', '').split(' ') if x])
                    except:
                        apart['metro'] = None
                    try:
                        apart['address'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--address-line--GRDTb").text.replace('На карте', '')
                    except:
                        apart['address'] = None
                    try:
                        apart['district'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--address-line--GRDTb").find_elements(by=By.TAG_NAME, value='a')[2].text.replace('р-н ', '')
                    except:
                        apart['district'] = None
                    try:
                        apart['rooms'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--title--vlZwT").text.split('-')[0]
                    except:
                        apart['rooms'] = None
                    try:
                        apart['total_area'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Общая площадь')]").find_element(by=By.XPATH, value="following-sibling::span").text.replace(' м²', '')
                    except:
                        apart['total_area'] = None

                    try:
                        apart['live_area'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Жилая площадь')]").find_element(by=By.XPATH, value="following-sibling::span").text.replace(' м²', '')
                    except:
                         apart['live_area'] = None
                    try:
                        apart['kitchen_area'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Площадь кухни')]").find_element(by=By.XPATH, value="following-sibling::span").text.replace(' м²', '')
                    except:
                        apart['kitchen_area'] = None
                    try:
                        apart['floor'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Этаж')]").find_element(by=By.XPATH, value="following-sibling::span").text.split(' из ')[0]
                        apart['max_floor'] = pages.find_element(by=By.XPATH, value="//span[contains(text(), 'Этаж')]").find_element(by=By.XPATH, value="following-sibling::span").text.split(' из ')[1]
                    except:
                        apart['floor'] = None
                        apart['max_floor'] = None
                    try:
                        apart['build_year'] = pages.find_element(by=By.XPATH, value="//p[contains(text(), 'Год постройки')]").find_element(by=By.XPATH, value="following-sibling::p").text
                    except:
                        apart['build_year'] = None
                    try:
                        apart['housing_type'] = pages.find_element(by=By.XPATH, value="//p[contains(text(), 'Тип жилья')]").find_element(by=By.XPATH, value="following-sibling::p").text
                    except:
                        apart['housing_type'] = None
                    try:
                        apart['meter_cost'] = pages.find_element(by=By.XPATH, value="//p[contains(text(), 'Стоимость метра')]").find_element(by=By.XPATH, value="following-sibling::p").text.replace(' ₽/м²', '').replace(' ', '')
                    except:
                        apart['meter_cost'] = None
                    try:
                        apart['bath'] = pages.find_element(by=By.XPATH, value="//p[contains(text(), 'Санузел')]").find_element(by=By.XPATH, value="following-sibling::p").text
                    except:
                        apart['bath'] = None
                    try:
                        apart['repair'] = pages.find_element(by=By.XPATH, value="//p[contains(text(), 'Ремонт')]").find_element(by=By.XPATH, value="following-sibling::p").text
                    except:
                        apart['repair'] = None
                    try:
                        apart['window'] = pages.find_element(by=By.XPATH, value="//p[contains(text(), 'Вид из окон')]").find_element(by=By.XPATH, value="following-sibling::p").text
                    except:
                        apart['window'] = None
                    try:
                        apart['ceiling'] = pages.find_element(by=By.XPATH, value="//p[contains(text(), 'Высота потолков')]").find_element(by=By.XPATH, value="following-sibling::p").text.replace(' м', '')
                    except:
                        apart['ceiling'] = None
                    try:
                        apart['description'] = pages.find_element(by=By.CLASS_NAME, value="a10a3f92e9--content--TjQir").text
                    except:
                        apart['description'] = None

                stats.append(apart)
            except:
                #raise MyException('Моё исключение')
                pass

        data = pd.DataFrame(stats)
        data.to_csv(filename)
        pages.quit()
        #pagination = driver.find_element(by=By.CLASS_NAME, value="_93444fe79c--pagination--VL341").find_element(by=By.TAG_NAME, value="a")
        #driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//nav[@class='_93444fe79c--pagination--VL341']//a[last()]"))))
        #driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "._93444fe79c--pagination--VL341 a:first-child"))))
        try:
            driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//ul[@class='_93444fe79c--pages-list--gsEUE']//following-sibling::a"))))
        except:
            break
        time.sleep(5)

    driver.quit()
    print(len(stats))

CPU times: total: 0 ns
Wall time: 0 ns


In [5]:
%%time
parse_cian('https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=16000000&minprice=14000000&object_type%5B0%5D=1&offer_type=flat&region=1&room2=1', 53, 'cian2_6.csv')
time.sleep(60)
parse_cian('https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=18000000&minprice=16000000&object_type%5B0%5D=1&offer_type=flat&region=1&room2=1', 53, 'cian2_7.csv')
time.sleep(60)
parse_cian('https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=21000000&minprice=18000000&object_type%5B0%5D=1&offer_type=flat&region=1&room2=1', 53, 'cian2_8.csv')
parse_cian('https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&maxprice=25000000&minprice=21000000&object_type%5B0%5D=1&offer_type=flat&region=1&room2=1', 53, 'cian2_9.csv')
time.sleep(60)
parse_cian('https://www.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&minprice=25000000&object_type%5B0%5D=1&offer_type=flat&region=1&room2=1', 53, 'cian2_10.csv')

1430
1431
1426
1428
1431
CPU times: total: 4min 15s
Wall time: 4h 32min 32s


In [7]:
data = pd.DataFrame(stats)
data.to_csv('cian.csv')